In [1]:
import numpy as np
import random
import copy
import sys
import math

In [2]:
num_features = 12
num_rows = 1000
seed = 42 # interesting seeds: 28, 32, (42), 56, 58, 63, 91
max_epochs = 1000 # for PSO algorithm use

In [3]:
random.seed(seed)

def make_all_data():
    weights = [20.0*random.random()-10.0 for i in range(num_features+1)]
    # weights = np.array(weights)
    result = [[0 for j in range(num_features+1)]
              for i in range(num_rows)]
    
    for i in range(num_rows):
        y = weights[0]
        for j in range(num_features):
            x = 20.0*random.random() - 10.0
            result[i][j] = x
            wx = x * weights[j+1] # weight * x
            y += wx
            y += num_features*random.random()
        if y > num_features:
            result[i][num_features] = 1.0
        else:
            result[i][num_features] = 0.0
    
    print("Data generation weights:")
    print(weights)
    
    return result

all_data = make_all_data()

Data generation weights:
[2.7885359691576745, -9.49978489554666, -4.499413632617615, -5.5357852370235445, 4.729424283280249, 3.533989748458225, 7.843591354096908, -8.261223347411677, -1.561563606294591, -9.404055611238594, -5.627240503927933, 0.10710576206724731, -9.469280606322727]


In [4]:
random.seed(seed)

def make_train_test():
    tot_rows = num_rows
    num_train_rows = int(tot_rows * 0.80) # 80% hard-coded
    num_test_rows = tot_rows - num_train_rows
    
    copy_data = copy.copy(all_data)
    random.shuffle(copy_data)
    
    train_data = [copy_data[i] for i in range(num_train_rows)]
    test_data = [copy_data[i+num_train_rows] for i in range(num_test_rows)]
    
    return train_data, test_data

train_data, test_data = make_train_test()
print("Training data:\n")
for i in range(3):
    print(train_data[i])
print("...\n")

print("Test data:\n")
for i in range(3):
    print(train_data[i])
print("...\n")

Training data:

[9.093842867305796, 8.582512498031448, -2.5154950969167533, -9.227612707342237, 5.56237520665616, 8.203223845987466, -1.2762408148880766, -3.6914724212230894, -6.43479135867405, -6.38399926947293, -3.3642956262958297, 2.7040454220050982, 1.0]
[3.6264345924366204, -9.864493165777846, 4.242274277577744, -7.6063323356776085, 2.277009198610296, 4.607606852234484, -5.024009349974701, 5.176420061229196, -6.375238145964213, -0.10049898495270071, -8.217885480201033, 1.480057791013806, 1.0]
[-9.924396467712373, 5.772048426322042, -5.731118221069183, 5.469136351103792, 2.508336264596334, 8.272219170373823, -0.6512116690185472, 8.784668903862272, 4.519531355236772, -9.324601746359226, 8.57159574311044, -9.434456985367374, 1.0]
...

Test data:

[9.093842867305796, 8.582512498031448, -2.5154950969167533, -9.227612707342237, 5.56237520665616, 8.203223845987466, -1.2762408148880766, -3.6914724212230894, -6.43479135867405, -6.38399926947293, -3.3642956262958297, 2.7040454220050982, 1.0

In [11]:
class Particle:
    def __init__(self, position, error, velocity, best_position, best_error):
        self.position = position # equivalent to weights
        self.error = error # measure of fitness
        self.velocity = velocity # determines new position
        self.best_position = best_position # best found by this Particle
        self.best_error = best_error

In [38]:
random.seed(seed)

class LogisticClassifier:
    def __init__(self):
        self.num_features = num_features
        self.weights = [0.0 for i in range(num_features+1)]
    
    def find_good_L1_weight(self):
        result = 0.0
        best_err = sys.float_info.max
        curr_err = sys.float_info.max
        candidates = [0.000, 0.001, 0.005, 0.010, 0.020, 0.050, 0.100, 0.150]

        c = LogisticClassifier()
        
        for trial in range(len(candidates)):
            alpha1 = candidates[trial]
            wts = c.train(alpha1, 0.0)
            curr_err = self.error(wts, 0.0, 0.0)
            if curr_err < best_err:
                best_err = curr_err
                result = candidates[trial]
        return result;
    
    def find_good_L2_weight(self):
        result = 0.0
        best_err = sys.float_info.max
        curr_err = sys.float_info.max
        candidates = [0.000, 0.001, 0.005, 0.010, 0.020, 0.050, 0.100, 0.150]

        c = LogisticClassifier()
        
        for trial in range(len(candidates)):
            alpha2 = candidates[trial];
            wts = c.train(0.0, alpha2)
            curr_err = self.error(wts, 0.0, 0.0)
            if curr_err < best_err:
                best_err = curr_err
                result = candidates[trial]

        return result;
    
    def train(self, alpha1, alpha2):
        # use PSO. particle position == LR weights
        num_particles = 10
        prob_death = 0.005
        dim = self.num_features + 1 # need one wt for each feature, plus the b0 constant
        
        epoch = 0
        minX = -10.0 # for each weight. assumes data has been normalized about 0
        maxX = 10.0
        w = 0.729 # inertia weight
        c1 = 1.49445 # cognitive/local weight
        c2 = 1.49445 # social/global weight
        r1, r2 = 0.0, 0.0 # cognitive and social randomizations

        swarm = [0.0 for i in range(num_particles)]
        # best solution found by any particle in the swarm. implicit initialization to all 0.0
        best_swarm_position = [0.0 for i in range(dim)]
        best_swarm_error = sys.float_info.max # smaller values better
        
        for i in range(len(swarm)):
            random_position = [0.0 for j in range(dim)]
            for j in range(len(random_position)):
                random_position[j] = (maxX - minX) * random.random() + minX
                
            # random_position is a set of weights
            error_ = self.error(random_position, alpha1, alpha2)
            random_velocity = [0.0 for i in range(dim)]
            for j in range(len(random_velocity)):
                lo, hi = 0.1 * minX, 0.1 * maxX
                random_velocity[j] = (hi - lo) * random.random() + lo
            
            # last two are best-position and best-error
            swarm[i] = Particle(random_position, error_, random_velocity, random_position, error_)

            # does current Particle have global best position/solution?
            if swarm[i].error < best_swarm_error:
                best_swarm_error = swarm[i].error
                best_swarm_position = copy.copy(swarm[i].position)
            # initialization     
            
        # main PSO algorithm
        sequence = [i for i in range(num_particles)] # process particles in random order
            
        for epoch in range(max_epochs):
            new_velocity = [0.0 for i in range(dim)] # step 1
            new_position = [0.0 for i in range(dim)] # step 2
            new_error = 0.0 # step 3
            random.shuffle(sequence) # move particles in random sequence
                
            for pi in range(len(swarm)): # each Particle (index)
                i = sequence[pi]
                currP = swarm[i] # for coding convenience

                # 1. compute new velocity
                for j in range(len(currP.velocity)): # each x value of the velocity
                    r1, r2 = random.random(), random.random()
                        
                    # velocity depends on old velocity, best position of parrticle, and 
                    # best position of any particle
                    new_velocity[j] = (w * currP.velocity[j]) + (c1 * r1 * (currP.best_position[j] - currP.position[j])) + (c2 * r2 * (best_swarm_position[j] - currP.position[j]))
                        
                currP.velocity = copy.copy(new_velocity)

                # 2. use new velocity to compute new position
                for j in range(len(currP.position)):
                    new_position[j] = currP.position[j] + new_velocity[j] # compute new position
                    if new_position[j] < minX: # keep in range
                        new_position[j] = minX
                    elif new_position[j] > maxX:
                        new_position[j] = maxX

                currP.position = copy.copy(new_position)

                # 3. use new position to compute new error
                new_error = self.error(new_position, alpha1, alpha2)
                currP.error = new_error

                if new_error < currP.best_error: # new particle best?
                    currP.best_position = copy.copy(new_position)
                    currP.best_error = new_error

                if (new_error < best_swarm_error): # new swarm best?
                    best_swarm_position = copy.copy(new_position)
                    best_swarm_error = new_error
                        
                # 4. optional: does curr particle die?
                die = random.random()
                if die < prob_death:
                    # new position, leave velocity, update error
                    for j in range(len(currP.position)):
                        currP.position[j] = (maxX - minX) * random.random() + minX
                    currP.error = self.error(currP.position, alpha1, alpha2)
                    currP.best_position = currP.position
                    currP.best_error = currP.error

                    if currP.error < best_swarm_error: # swarm best by chance?
                        best_swarm_error = currP.error
                        best_swarm_position = copy.copy(currP.position)
        
        ret_result = copy.copy(best_swarm_position)
        return ret_result
    
    def accuracy(self, data, weights):
        num_correct = 0
        num_wrong = 0
        y_index = len(data[0]) - 1
        
        for i in range(len(data)):
            computed = self.compute_dependent(data[i], weights) # implicit cast
            desired = data[i][y_index] # 0.0 or 1.0

            epsilon = 0.0000000001
            if math.fabs(computed - desired) < epsilon:
                num_correct += 1
            else:
                num_wrong += 1
        return (num_correct * 1.0) / (num_wrong + num_correct)
    
    def error(self, weights, alpha1, alpha2):
        # mean squared error using supplied weights
        # L1 regularization adds the sum of the absolute values of the weights
        # L2 regularization adds the sqrt of sum of squared values

        y_index = len(train_data[0]) - 1 # y-value (0/1) is last column
        sum_squared_error = 0.0
        for i in range(len(train_data)):
            computed = self.compute_output(train_data[i], weights)
            desired = train_data[i][y_index] # ex: 0.0 or 1.0
            sum_squared_error += (computed - desired) * (computed - desired)

        sum_abs_vals = 0.0 # L1 penalty
        for i in range(len(weights)):
            sum_abs_vals += math.fabs(weights[i])

        sum_squared_vals = 0.0 # L2 penalty
        for i in range(len(weights)):
            sum_squared_vals += (weights[i] * weights[i])
        # root_sum = math.sqrt(sum_squared_vals)

        return (sum_squared_error / len(train_data)) + (alpha1 * sum_abs_vals) + (alpha2 * sum_squared_vals)
    
    def compute_output(self, data_item, weights):
        z = 0.0

        z += weights[0] # the b0 constant
        for i in range(len(weights)-1): # data might include Y
            z += (weights[i + 1] * data_item[i]) # skip first weight
        return 1.0 / (1.0 + math.exp(-z))
    
    def compute_dependent(self, data_item, weights):
        sum_ = self.compute_output(data_item, weights)

        if sum_ <= 0.5:
            return 0
        else:
            return 1

print("Creating LR binary classifier")
lc = LogisticClassifier()
print("Starting training using no regularization")
weights = lc.train(0.0, 0.0)

print("Best weights found:")
print(weights)

Creating LR binary classifier
Starting training using no regularization
Best weights found:
[10.0, -10.0, -10.0, -3.856539591775924, 5.36017129115427, 5.197936689878941, 10.0, -10.0, -1.1583973536907237, -10.0, -9.801043414185703, 0.4598789216573807, -10.0]


In [35]:
train_accuracy = lc.accuracy(train_data, weights)
print("Prediction accuracy on training data = %f" % train_accuracy)

test_accuracy = lc.accuracy(test_data, weights)
print("Prediction accuracy on test data = %f" % test_accuracy)

Prediction accuracy on training data = 0.861250
Prediction accuracy on test data = 0.860000


In [36]:
print("Seeking good L1 weight")
alpha1 = lc.find_good_L1_weight()
print("Good L1 weight = %f" % alpha1)

Seeking good L1 weight
Good L1 weight = 0.005000


In [37]:
print("Starting training using L1 regularization, alpha1 = %f" % alpha1)
weights = lc.train(alpha1, 0.0)
print("Best weights found:")
print(weights)

train_accuracy = lc.accuracy(train_data, weights)
print("Prediction accuracy on training data = %f" % train_accuracy)

test_accuracy = lc.accuracy(test_data, weights)
print("Prediction accuracy on test data = %f" % test_accuracy)

Starting training using L1 regularization, alpha1 = 0.005000
Best weights found:
[10.0, -1.3449764559269253, -0.5651064344441007, -0.8314225013136549, 0.6150782132992475, 0.4569079933190636, 1.1203958418682936, -1.0754684033711657, -0.24530382855259114, -1.3605053709213448, -0.8246044782643088, 1.6227453222244882e-14, -1.3093217936868782]
Prediction accuracy on training data = 0.973750
Prediction accuracy on test data = 0.955000


In [39]:
print("Seeking good L2 weight")
alpha2 = lc.find_good_L2_weight()
print("Good L2 weight = %f" % alpha2)

Seeking good L2 weight
Good L2 weight = 0.001000


In [40]:
print("Starting training using L2 regularization, alpha1 = %f" % alpha2)
weights = lc.train(0.0, alpha2)
print("Best weights found:")
print(weights)

train_accuracy = lc.accuracy(train_data, weights)
print("Prediction accuracy on training data = %f" % train_accuracy)

test_accuracy = lc.accuracy(test_data, weights)
print("Prediction accuracy on test data = %f" % test_accuracy)

Starting training using L2 regularization, alpha1 = 0.001000
Best weights found:
[3.0117250873983097, -0.4782303023615921, -0.2146089041710627, -0.2695893928788065, 0.24627049656117098, 0.19086912395983371, 0.4147897975541401, -0.4245648558506859, -0.08231130086269411, -0.5110140032755235, -0.3035986751866559, 0.02509893222696457, -0.4708246452708215]
Prediction accuracy on training data = 0.971250
Prediction accuracy on test data = 0.945000


#### References

1. [Particle Swarm Optimization: A Tutorial - Armstrong](http://www.cs.armstrong.edu/saad/csci8100/pso_tutorial.pdf), by James Blondin
2. [What is the difference between L1 and L2 regularization? - Quora](https://www.quora.com/What-is-the-difference-between-L1-and-L2-regularization)
3. [Test Run - L1 and L2 Regularization for Machine Learning](https://msdn.microsoft.com/en-us/magazine/dn904675.aspx), by James McCaffrey
4. [《统计学习方法》，李航著](https://book.douban.com/subject/10590856/)